In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/RAG/requirements.txt"

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ozd9wd8n
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ozd9wd8n
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from langchain_openai.chat_models import ChatOpenAI
OPENAI_API_KEY ="sk-proj-Re7WpnPtgygJewBx6cNdT3BlbkFJdnVWCv1sGZhbmcmqOGJv"
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4")

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Given the movies data and corresponding book titles in the user context {context}, your task is to select the most relevant book title from the provided {data} based on the movie review. Provide only the selected book title as the output. Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/RAG/train")
#df1=df[["title","text","Target_Title"]]
df1=df.rename(columns={"Target_Title":"Book Title"})
df1.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,Book Title
0,5.0,Great movie!,Mark Wahlberg was great! The movie was poignan...,[],B09VT2M351,B09VT2M351,AE223GHNZEI5MRMBVVRGJONDNWRQ,2022-08-02 05:09:35.884,0,True,Golf Heaven
1,3.0,Rated 4.5 stars - should be 3.5,This movie was a little drawn out and predicta...,[],B09ZFBBSC6,B09ZFBBSC6,AE223GHNZEI5MRMBVVRGJONDNWRQ,2022-08-02 05:13:17.188,1,True,Ultimate Guide to Golf
2,4.0,Great movie!,Crazy these stories are true! Great movie,[],B0B19ZNXXH,B0B19ZNXXH,AE223GHNZEI5MRMBVVRGJONDNWRQ,2022-08-02 05:16:36.140,0,True,Words can Hurt or Heal
3,5.0,timeless,It's a classic beyond compare. I like this par...,[],B003CSV7TK,B003CSV7TK,AE224GVO7OHTYF26U6ER6BEVIUAQ,2014-11-27 14:19:26,0,True,"An easy way to acquire ""Zils"""
4,1.0,disgracing many good figures of our nation's h...,"This movie, based or un-based on actual events...",[],B00EV4EUT8,B00EV4EUT8,AE224GVO7OHTYF26U6ER6BEVIUAQ,2015-06-27 23:44:10,1,False,Inspired Creativity


In [ ]:
columns_to_drop=['images']
df1.fillna(value=pd.NA, inplace=True)
df1.dropna(inplace=True)
df1.drop(columns=columns_to_drop, inplace=True)

In [ ]:
#df['review'] = df['title'] + ' ' + df['text']

In [ ]:
df2=df1.head(20)

In [ ]:
df2.to_csv("/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train3")

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train1", csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'Book Title']
})
#loader = CSVLoader("/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train")
text_documents = loader.load()
text_documents[:10]

[Document(page_content='rating: \ntitle: rating\ntext: title\nimages: text\nasin: asin\nparent_asin: parent_asin\nuser_id: user_id\ntimestamp: timestamp\nhelpful_vote: helpful_vote\nverified_purchase: verified_purchase\nBook Title: Book Title', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train1', 'row': 0}),
 Document(page_content='rating: 0\ntitle: 5.0\ntext: Great movie!\nimages: Mark Wahlberg was great! The movie was poignant and is a must see!\nasin: B09VT2M351\nparent_asin: B09VT2M351\nuser_id: AE223GHNZEI5MRMBVVRGJONDNWRQ\ntimestamp: 2022-08-02 05:09:35.884\nhelpful_vote: 0\nverified_purchase: True\nBook Title: Golf Heaven', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train1', 'row': 1}),
 Document(page_content='rating: 1\ntitle: 3.0\ntext: Rated 4.5 stars - should be 3.5\nimages: This movie was a little drawn out and predictable, I was hoping for a little more.\nasin: B09ZFBBSC6\nparent_asin: B09ZFBBSC6\nuser_id: AE223GHN

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [ ]:
import pandas as pd
df3=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/RAG/test")

In [ ]:
df4=df3.drop(columns=["Target_Title"])


In [ ]:
df4.head(20)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,3.0,Okay Movie,"Very predictable, and drawn out",[],B0044WZNB6,B0044WZNB6,AE223GHNZEI5MRMBVVRGJONDNWRQ,2022-08-02 05:20:10.721,0,True
1,5.0,One of the best!,Love this season. Great storyline. Glad the Ha...,[],B07GCQG6W5,B07GCQG6W5,AE224GVO7OHTYF26U6ER6BEVIUAQ,2018-08-19 03:46:48.789,3,False
2,5.0,Five Stars,Can't go wrong with Swayze and Reed,[],B00932AGHI,B00932AGHI,AE22CFXT3QZKUQJORVTGL3VQXAAA,2018-06-12 13:54:09.487,0,True
3,4.0,Four Stars,Love Batman/Superman-what can I say,[],B077H5GFZS,B077H5GFZS,AE22CFXT3QZKUQJORVTGL3VQXAAA,2018-06-12 13:54:30.754,0,True
4,5.0,Five Stars,Love it,[],B01J4STMP2,B01J4STMP2,AE22CS5OIRQTN7WDKQ2F3BYP5CXA,2016-12-10 09:44:54,0,True
5,5.0,Five Stars,Loved first season and loving the second,[],B01IS2PWV8,B01IS2PWV8,AE22CS5OIRQTN7WDKQ2F3BYP5CXA,2017-01-31 19:42:04,0,True
6,5.0,Wonderful movie,I watched this movie for a film discussion gro...,[],B002J8VO0G,B002J8VO0G,AE236YP3ZIRW4LLJQXTMQUTE3Y2Q,2021-06-25 22:20:06.833,2,True
7,3.0,O. K. movie.,O.K. movie.,[],B008JFUR10,B008JFUR10,AE24FFSUQHE3J6NYBICB7V2WHUAA,2016-02-03 14:34:54,0,True
8,5.0,Black Panther - Just Wow!,I saw this movie in 3D in the theater but I th...,[],B01D9EUNBY,B01D9EUNBY,AE24FFSUQHE3J6NYBICB7V2WHUAA,2016-08-21 07:07:23,0,False
9,5.0,I highly recommend.,"It's an outstanding stretching video, I highly...",[],B00DYZ80L4,B00DYZ80L4,AE24FFSUQHE3J6NYBICB7V2WHUAA,2016-09-17 19:38:45,0,True


In [ ]:
data1=list(df3["Target_Title"][:10])

In [ ]:
data1

['Paradoxical Commandments',
 'Creative Thoughts About A Challenging Life',
 'A Cozy Read filled with Wisdom',
 'Tap into Secret Wealth',
 'Yet another enjoyable novel by Phyllis Whitney that brings back memories',
 'Sure to cure the winter blues....',
 'Inspirational Thoughts and Dreamy Pictures',
 'Analyzing and Addressing Pertinent Questions of Interest',
 'Authentic Leadership',
 'A Beautiful Book that Could Change the World']

In [ ]:
str(df4.iloc[1])

'rating                                                             5.0\ntitle                                                 One of the best!\ntext                 Love this season. Great storyline. Glad the Ha...\nimages                                                              []\nasin                                                        B07GCQG6W5\nparent_asin                                                 B07GCQG6W5\nuser_id                                   AE224GVO7OHTYF26U6ER6BEVIUAQ\ntimestamp                                      2018-08-19 03:46:48.789\nhelpful_vote                                                         3\nverified_purchase                                                False\nName: 1, dtype: object'

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

parser = StrOutputParser()
retriever=vectorstore2.as_retriever()
import random

def generate_label_subsets(labels, num_instances, num_incorrect_labels):
    """
    Generate label subsets for each instance.

    Args:
    - labels (list): List of labels.
    - num_instances (int): Number of instances.
    - num_incorrect_labels (int): Number of incorrect labels per instance.

    Returns:
    - list: List of lists, each containing the correct label followed by a list of incorrect labels.
    """
    label_subsets = []

    for _ in range(num_instances):
        correct_label = random.choice(labels)
        incorrect_labels = random.sample([label for label in labels if label != correct_label], num_incorrect_labels)
        subset = [correct_label] + incorrect_labels
        random.shuffle(subset)  # Shuffle the subset
        label_subsets.append(subset)

    return label_subsets

# Example usage
labels =data1
num_instances = len(data1)
num_incorrect_labels = 3


def create_list(*elements):
    """
    Create a list from the given elements.

    Args:
        *elements: Variable number of elements to be added to the list.

    Returns:
        List containing the input elements.
    """
    return list(elements)


label_subsets = generate_label_subsets(labels, num_instances, num_incorrect_labels)
for i in range(0,10):
  chain = (
    #{"context":retriever,"data":lambda x : ["Paradoxical Commandments","Creative Thoughts About A Challenging Life","A Cozy Read filled with Wisdom","Tap into Secret Wealth"],"question": RunnablePassthrough() }
     {"context":retriever,"data":lambda x :label_subsets[i],"question": RunnablePassthrough() }
    # {"context":retriever,"data":{"text1":itemgetter("text1"),"text2":itemgetter("text2"),"text3":itemgetter("text3"),"text4":itemgetter("text4")}|RunnableLambda(create_list),"question": itemgetter("question") }
    #  {"context":retriever,"data":{"text1":"Paradoxical Commandments","text2":"Creative Thoughts About A Challenging Life","text3": "A Cozy Read filled with Wisdom","text4":"Tap into Secret Wealth"}|RunnableLambda(create_list),"question": RunnablePassthrough() }
    | prompt
    | model
    | parser )
for i in range(10):
  print (f"Prediction{i}.{chain.invoke(str(df4.iloc[i]))}")
  print (f" Actual{i}.{df3['Target_Title'][i]}")

Prediction0.'Creative Thoughts About A Challenging Life'
 Actual0.Paradoxical Commandments
Prediction1.'A Cozy Read filled with Wisdom'
 Actual1.Creative Thoughts About A Challenging Life
Prediction2.A Cozy Read filled with Wisdom
 Actual2.A Cozy Read filled with Wisdom
Prediction3.'Sure to cure the winter blues....'
 Actual3.Tap into Secret Wealth
Prediction4.'Inspirational Thoughts and Dreamy Pictures'
 Actual4.Yet another enjoyable novel by Phyllis Whitney that brings back memories
Prediction5.'Sure to cure the winter blues....'
 Actual5.Sure to cure the winter blues....
Prediction6.'A Beautiful Book that Could Change the World'
 Actual6.Inspirational Thoughts and Dreamy Pictures
Prediction7.'Analyzing and Addressing Pertinent Questions of Interest'
 Actual7.Analyzing and Addressing Pertinent Questions of Interest
Prediction8.'A Beautiful Book that Could Change the World'
 Actual8.Authentic Leadership
Prediction9.'A Beautiful Book that Could Change the World'
 Actual9.A Beautiful Bo

In [ ]:
data1

['Paradoxical Commandments',
 'Creative Thoughts About A Challenging Life',
 'A Cozy Read filled with Wisdom',
 'Tap into Secret Wealth',
 'Yet another enjoyable novel by Phyllis Whitney that brings back memories',
 'Sure to cure the winter blues....',
 'Inspirational Thoughts and Dreamy Pictures',
 'Analyzing and Addressing Pertinent Questions of Interest',
 'Authentic Leadership',
 'A Beautiful Book that Could Change the World']